In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import datetime
import seaborn as sns

In [2]:
#import the Solar Panel dataframe
dfSP = pd.read_csv('Solar_Ninja_Data_1kW.csv')
dfSP['time'] = pd.to_datetime(dfSP['time'])

In [3]:
#import the OldBod dataframe
dfOB = pd.read_csv("SortedOldBodHHElectricity.csv",)
dfOB['Date'] = pd.to_datetime(dfOB['Date'], format='%d/%m/%Y')



#import the RadCam dataframe
dfRC = pd.read_csv("SortedRadCamHHElectricity.csv",)
dfRC['Date'] = pd.to_datetime(dfRC['Date'], format='%d/%m/%Y')



#drop the date column temporarily so we can add the two dataframes together
Dates = dfOB['Date']
dfRC = dfRC.drop('Date',  axis=1)
dfOB = dfOB.drop('Date',  axis=1)

#apply effiency savings of 5.7% for OldBod and 3.4% for the Radcam
dfOB = dfOB.multiply(0.943)
dfRC = dfRC.multiply(0.966)

#add them together
df = dfRC.add(dfOB, fill_value=0)


#reinsert the dates
df.insert(0, 'Date', Dates)
df = df[df['Date'].dt.year == 2017] 

In [4]:
#insert relevant columns for solar panels
dfSP['Date']=dfSP['time'].dt.date
dfSP['Month']=dfSP['time'].dt.month_name()
dfSP['Day']=dfSP['time'].dt.day_name()
dfSP['Hour']=dfSP['time'].dt.hour


#sort the dataframe columns in a desired order
dfSP = dfSP[['Month','Hour','electricity']]

#sub-dataframe of electrcity output of solar panels in summer months
df_Summer_SP = dfSP.loc[dfSP['Month'].isin(['June', 'July', 'August', 'September'])]
df_Summer_SP_Elec = df_Summer_SP['electricity']
df_Summer_SP_Elec = df_Summer_SP_Elec.reset_index()

#Sum every 24 hours to find the daily total outputs from solar panels
df_SP_Sum = df_Summer_SP_Elec.groupby(df_Summer_SP_Elec.index // 24).sum()
df_SP_Sum = df_SP_Sum['electricity']
df_SP_Sum = df_SP_Sum.multiply(250) #Size of plant in kW
df_SP_Sum = df_SP_Sum.div(1000) #Change Units to MWh

In [5]:
#insert relevant columns for libraries

df['Month'] = df['Date'].dt.month_name()
df['Day'] = df['Date'].dt.day_name()
df['Date']=df['Date'].dt.date
df['Daily Sum'] = df.sum(axis=1)

#sort the dataframe columns in a desired order
df = df[['Month','Day','Daily Sum']]

#sub-dataframe of libraries in summer months then isolate daily-sum column
df_Summer = df.loc[df['Month'].isin(['June', 'July', 'August', 'September'])]
Daily_Sum = df_Summer['Daily Sum']
Daily_Sum = Daily_Sum.reset_index()
Daily_Sum = Daily_Sum.div(1000) #Change Units to MWh
Daily_Sum = Daily_Sum['Daily Sum']

In [6]:
#Plot of Energy Generated by Swindon Solar Farm vs Energy Consumed by Radcliffe Camera and Old Bodleian Library
ax1 = df_SP_Sum.plot(label='Solar Generation')
Daily_Sum.plot(ax=ax1, label='Bodleian Consumption')
ax1.set_xlabel("Days in Summer")
ax1.set_ylabel("Daily Energy Generation/Consumption (MWh)")
plt.legend(framealpha=1, frameon=True);


In [7]:
# Excess/Deficit Energy Plot
Diff = df_SP_Sum - Daily_Sum
ax2 = Diff.plot()
ax2.set_xlabel("Days in Summer")
ax2.set_ylabel("Daily Excess Energy (MWh)")


Text(0, 0.5, 'Daily Excess Energy (MWh)')